In [ ]:
import PyPDF2

with open("U-Net.pdf", "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

import re

cleaned_text = re.sub(r"\s+", " ", text).strip() 

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")  
embeddings = model.encode(cleaned_text.split("."))  
vectorstore = dict(zip(cleaned_text, embeddings))
import pickle

with open("vectorstore2.pkl", "wb") as f:
    pickle.dump(vectorstore, f)



with open("/kaggle/input/vbusingdic2/vectorstore2.pkl", "rb") as f:
    vectorstore = pickle.load(f)


from langchain.prompts import PromptTemplate
from transformers import pipeline
import torch

prompt_template = """You are a helpful researcher that explains from your research work to other students.

{context}

Question: {question}
Answer here:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

instruct_pipeline = pipeline(
    model="databricks/dolly-v2-7b", 
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    return_full_text=True,
)

from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

llm = HuggingFacePipeline(pipeline=instruct_pipeline)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
)